# MLP2
## Jonathan Doan & Jesse McQuillan

In [1]:
# Common imports
import numpy as np
import pandas as pd
import re
import itertools

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# to make this notebook's output stable across runs
np.random.seed(42)

# a) Datasets

### Tweets.csv
A dataset containing tweets scraped from twitter regarding users wordle games<br>
----------columns---------- <br>
wordle_id: Unique id for each daily wordle game<br>
tweet_id: Unique id for each individual tweet<br>
tweet_date: self-explanatory<br>
tweet_username: self-explanatory<br>
tweet_text: self-explanatory<br>
<br>
Contains 410,186 tweets with 5 attributes

In [2]:
wordleTweets = pd.read_csv("../rawdata/tweets.csv")
wordleTweets.head(20)
wordleTweets.tail()

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
410182,291,1511736826435878913,2022-04-06 16:05:20+00:00,SpottedLights,Wordle 291 4/6\n\n🟨⬛⬛⬛⬛\n⬛⬛🟨⬛⬛\n🟩⬛🟨🟨⬛\n🟩🟩🟩🟩🟩
410183,291,1511736827195011072,2022-04-06 16:05:20+00:00,AlisonB916,Wordle 291 4/6\n\n⬜🟩⬜⬜⬜\n⬜⬜🟨⬜⬜\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩
410184,291,1511736850741878784,2022-04-06 16:05:26+00:00,Nico19596830,Wordle 291 4/6\n\n🟩⬛⬛🟨🟨\n🟩🟨🟩⬛⬛\n⬛🟩⬛⬛⬛\n🟩🟩🟩🟩🟩
410185,291,1511736856433631233,2022-04-06 16:05:27+00:00,RumzHere,Wordle 291 3/6\n\n🟩⬜🟨🟩⬜\n🟩🟨🟩⬜🟨\n🟩🟩🟩🟩🟩
410186,291,1511736863987490826,2022-04-06 16:05:29+00:00,weboggle,Wordle 291 4/6\n\n⬜⬜⬜🟨⬜\n🟩🟩⬜⬜⬜\n🟩🟩⬜⬜🟩\n🟩🟩🟩🟩🟩\n🌃


### unigram_freq.csv
'This dataset contains the counts of the 333,333 most commonly-used single words on the English language web, as derived from the Google Web Trillion Word Corpus.'<br>
----------columns---------- <br>
word: the word in question<br>
count: relative frequency of a word<br>
<br>
Contains 333,333 words with 2 attributes

In [66]:
word_freqs = pd.read_csv("../rawdata/unigram_freq.csv")
# make sure you understand the type of the object
print(type(word_freqs))

print(word_freqs.describe(),'\n')
# check the top five and the bottom five data tuples
display(word_freqs.head())
display(word_freqs.tail())
display(word_freqs.shape)
#check for nulls
display(word_freqs[word_freqs.isna().any(axis=1)])

<class 'pandas.core.frame.DataFrame'>
              count
count  3.333330e+05
mean   1.764374e+06
std    6.629987e+07
min    1.271100e+04
25%    2.122400e+04
50%    4.151900e+04
75%    1.365760e+05
max    2.313585e+10 



,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


,word,count
333328,gooek,12711
333329,gooddg,12711
333330,gooblle,12711
333331,gollgo,12711
333332,golgw,12711


(333333, 2)

,word,count
2577,NaN,30739157
12819,NaN,3398089


### answers.csv
'This dataset contains every wordle answer in order'<br>
----------columns---------- <br>
word: the word in question<br>
<br>
Contains 2315 words

In [39]:
#Get data locally
df_answers = pd.read_csv("../rawdata/answers.csv", header=None)
df_answers.columns = ['word']

#look at data
display(df_answers.head())
display(df_answers.tail())
display(df_answers.shape)
display(df_answers.describe())
print(type(df_answers))
answers = df_answers.astype({"word": str}, errors='raise') 
print(df_answers.dtypes)

,word
0,cigar
1,rebut
2,sissy
3,humph
4,awake


,word
2310,judge
2311,rower
2312,artsy
2313,rural
2314,shave


(2315, 1)

,word
count,2315
unique,2315
top,cigar
freq,1


<class 'pandas.core.frame.DataFrame'>
word    object
dtype: object


### other_words.csv
'This dataset contains every acceptable wordle guess that isn't an answer'<br>
----------columns---------- <br>
word: the word in question<br>
<br>
Contains 10,657 words

In [6]:
other_words = pd.read_csv("../rawdata/other_words.csv", header=None)
other_words.columns = ['word']
display(other_words.head())
display(other_words.tail())
display(other_words.shape)
display(other_words.describe())
other_words = other_words.astype({"word": str}, errors='raise') 
print(type(other_words))

,word
0,aahed
1,aalii
2,aargh
3,aarti
4,abaca


,word
10652,zuzim
10653,zygal
10654,zygon
10655,zymes
10656,zymic


(10657, 1)

,word
count,10657
unique,10657
top,aahed
freq,1


<class 'pandas.core.frame.DataFrame'>


# b) Data Pre-Processing

#### Cleaning tweets

In [7]:
#sort them by wordle id to make prevent out of sequence errors
wordleTweets.sort_values(by=['wordle_id'], inplace=True)

games = []
daily = []
cur_id = 0

#process each tweet to make text machine readable
for index, tweet in wordleTweets.iterrows():
    
    #Set first index
    if index == 0:
        cur_id = tweet.wordle_id
        
    #find all guess emoju strings
    raw_guesses = re.findall("([🟩🟨⬛⬜]+)", tweet.tweet_text)
    single_game = []
    
    #replace emojs with letter
    for guess in raw_guesses:
        guess = guess.replace("🟩", 'G')
        guess = guess.replace("🟨", 'Y')
        guess = guess.replace("⬛", 'B')
        guess = guess.replace("⬜", 'B')
        single_game.append(guess)
    #print(single_game)
    
    #add to the days games
    daily.append(single_game[:])
    
    #add the day to the list and start a new daily
    if tweet.wordle_id != cur_id:
        #print(len(daily))
        games.append(daily[:])
        daily.clear()
        cur_id = tweet.wordle_id

print("Complete!")

Complete!


#### Cleaning word frequencies

In [73]:
#the two words that didn't process correctly were 'null' and 'nan' and since those aren't five letterswe'll just drop them

word_freqs.dropna(inplace=True)

print(word_freqs.isnull().sum(), '\n')

word     0
count    0
dtype: int64 



### Making a master word list

In [41]:
#Prepare to merge word lists
frames = [df_answers, other_words]

#for some reason this one turned out uppercase.
df_answers.iloc[1345] = df_answers.iloc[1345].str.lower()


#merge and sort
df_words = pd.concat(frames, ignore_index=True)
df_words.sort_values(by=['word'], inplace=True)

display(df_words.head())
display(df_words.tail())


,word
2315,aahed
2316,aalii
2317,aargh
2318,aarti
2319,abaca


,word
12967,zuzim
12968,zygal
12969,zygon
12970,zymes
12971,zymic


In [78]:
df_master = df_words.join(word_freqs.set_index('word'), on='word')
display(df_master_og.isna().sum())

df_answers = answers.join(word_freqs.set_index('word'), on='word')
display(df_answers.isna().sum())

df_other_words = other_words.join(word_freqs.set_index('word'), on='word')
display(df_other_words.isna().sum())

word        0
count    4880
dtype: int64

word     0
count    2
dtype: int64

word        0
count    4879
dtype: int64

In [80]:
#df_master.set_index('word', inplace=True)

#The words with no frequency valid guesses but not words people actually use. 
#Assign them a count equal to the lowest value from the main list (which is slightly lower than the lowest value in the joined list)
df_master['count'].fillna(word_freqs['count'].min(), inplace=True)
df_answers['count'].fillna(word_freqs['count'].min(), inplace=True)
df_other_words['count'].fillna(word_freqs['count'].min(), inplace=True)

#Normalizing the count frequencies
df_master['normalized_count'] = (df_master['count'] - df_master['count'].min()) / (df_master['count'].max() - df_master['count'].min())
df_other_words['Normalized_count'] = (df_other_words['count'] - df_other_words['count'].min()) / (df_other_words['count'].max() - df_other_words['count'].min())
df_answers['Normalized_count'] = (df_answers['count'] - df_answers['count'].min()) / (df_answers['count'].max() - df_answers['count'].min())

#Sort by count
df_master.sort_values(by=['count'], inplace=True)
df_other_words.sort_values(by=['count'], inplace=True)
df_answers.sort_values(by=['count'], inplace=True)

#reset the indexes
df_master.reset_index(drop=True, inplace=True)
df_other_words.reset_index(drop=True, inplace=True)
df_answers.reset_index(drop=True, inplace=True)

#show the work
display(df_master.head())
display(df_other_words.head())
display(df_answers.head())


,word,count,normalized_count
0,aahed,12711.0,0.0
1,fabby,12711.0,0.0
2,eyrir,12711.0,0.0
3,pelau,12711.0,0.0
4,eyras,12711.0,0.0


,word,count,Normalized_count
0,aahed,12711.0,0.0
1,murid,12711.0,0.0
2,mured,12711.0,0.0
3,muras,12711.0,0.0
4,muntu,12711.0,0.0


,word,count,Normalized_count
0,wooer,12711.0,0.000000
1,FALSE,12711.0,0.000000
2,elate,16161.0,0.000003
3,vaunt,17134.0,0.000004
4,snaky,18580.0,0.000005


# c) Exploratory analysis

### A list of tile combinations for computer probabilities of guesses

In [11]:
#Create a lit of every possible tile combination
unique_tiles = ['G', 'Y', 'B']
impossible_tiles = ['GGGGG', 'GGGGY', 'GGGYG', 'GGYGG', 'GYGGG', 'YGGGG', ]
tile_combos = []

for raw_answer in itertools.product(unique_tiles, repeat=5):
    clean_answer = ''.join(raw_answer)
    if clean_answer not in impossible_tiles:
        tile_combos.append(''.join(raw_answer))

#print (tile_combos)

### functions for getting guess probability distributions

In [12]:
#Takes in a word and returns a distribution of probably answers based on how many words correspond to a particular tile response
word_size = 5
new_line = '\n'
GREEN = 'G'
YELLOW = 'Y'
BLACK = 'B'

#Takes in a word and a set of tiles, and returns a list of all valid five letter words given the conditions
def get_possible_answers(word, tiles):
    re_exp = ""
    remaining_words = df_words
    #print(len(remaining_words))
    #print(tiles)
    for i in range(word_size):
        if tiles[i] == GREEN:
            for j in range(word_size):
                if i == j:
                    re_exp += word[i]
                if i != j:
                    re_exp += '.'
        if tiles[i] == YELLOW:
            more_re = re.compile(r'(?=[a-z]*[' + word[i] + '])[a-z]{5}')
            remaining_words = list(filter(more_re.search, remaining_words))
            for j in range(word_size):
                if j == 0:
                    re_exp += '^'
                if i == j:
                    re_exp += '[^' + word[i] + ']'
                if i != j:
                    re_exp += '.'
        if tiles[i] == BLACK:
            for j in range(word_size):
                if j == 0:
                    re_exp += '^'
                if i == j:
                    re_exp += '[^' + word[i] + ']'
                if i != j:
                    re_exp += '.'
        
        #print(re_exp)
        remaining_words = list(filter(re.compile(re_exp).search, remaining_words)) 
        #print(remaining_words)
        re_exp = ''
    #print(remaining_words_list)
    #print('words found')
    return remaining_words
    

# takes in a word and returns a dictionary tile combinations as keys and lists of possible words as values
def get_guess_word_distro(word):
    answer_dict = {}
    for combo in tile_combos:
        remaining_possibilities = get_possible_answers(word, combo)
        answer_dict[combo] = remaining_possibilities
    #print('distro found')
    return answer_dict

#converts a pattern, word dictionary to a sorted pandas dataframe with counts of how many words correspond with each pattern
def word_to_count_distro(guess_words_dict):
    cols = ['pattern','count']
    df_guess_dist = pd.DataFrame(data=None, columns=cols)
    for tiles, guesses in guess_words_dict.items():
        #print(tiles, len(guesses))
        data = [tiles, len(guesses)]
        df_guess_dist.loc[len(df_guess_dist.index)+1] = data
    df_guess_dist.set_index('pattern')
    df_guess_dist['count'] = pd.to_numeric(df_guess_dist["count"])
    df_guess_dist.sort_values(by=['count'], inplace=True)
    return df_guess_dist


### Test me out! insert a different word below and see a (very rough) graph of the probability distributions of all possible tile outcomes

In [81]:
check_my_distribution = 'crane'

guess_words_dict = get_guess_word_distro(check_my_distribution)

df_distro = word_to_count_distro(guess_words_dict)

df_distro.plot()



NameError: name 'words' is not defined

### Combine the words dictionary and frequencies to see the most frequent answers in terms of usage

In [ ]:
#Joining the dataset together
df_other = other_word.join(frequency_alpha.set_index('word'), on='word')
df_ans = answers.join(word_freqs.set_index('word'), on='word')

#Normalizing the count frequencies
df_other['Normalized_count'] = (df_other['count'] - df_other['count'].min()) / (df_other['count'].max() - df_other['count'].min())
df_ans['Normalized_count'] = (df_ans['count'] - df_ans['count'].min()) / (df_ans['count'].max() - df_ans['count'].min())

print('Complete!')

###  hmmm
There appear to be some nans we'll have to deal with. I have my eyes on another frequency list that I have on good authority will cover all wordle words so that's coming.

In [ ]:
display(df_ans.head(20))
print(df_ans.isnull().sum(), '\n')

display(df_other.head(20))
print(df_other.isnull().sum(), '\n')

In [ ]:
#Mostly debugging code but I didn't think it should get thrown away yet

#word = 'grape'
#i = 2
#more_re = '(?![a-z]*[' + word[i] + '])[a-z]{5})'
#print (more_re)
#contains = 'al'
#contains_re = '(?=[a-z]*[{0}])'.format(contains)
#disallowed = 'h'
#disallowed_re = '(?![a-z]*[{0}])'.format(disallowed)
#match = 'tr'
#word_size = 5
#match_re = '[a-z]{'+ str(word_size - len(match)) + '}'
#final_re = match + contains_re + disallowed_re + match_re
#print(final_re)

#new_line = '\n'
#remaining_words = new_line.join(words)
#print (remaining_words)
#more_re = r'^[^b]....$'
#results = re.findall(more_re, remaining_words)
#print(len(results))
#print(results)
#print(new_line.join(results))

#test = ['blabs', 'black', 'blade', 'blads', 'blady', 'blaer', 'blaes', 'blaff', 'blags', 'blahs', 'blain', 'blame', 'blams', 'bland', 'blank', 'blare', 'blart', 'blase', 'blash', 'blast', 'blate', 'blats', 'blatt', 'blaud', 'blawn', 'blaws', 'blays', 'blaze']
#temp_re = re.compile(r'^....s')

#results = list(filter(temp_re.search, test)) 
#print(len(results))
#print(results)


#Read answers
#answers_file = open("../rawdata/answers.txt")
#raw_answers = answers_file.read()
#answers = raw_answers.splitlines()
#print(answers[:10])

#Read other words
#other_words_file = open("../rawdata/other_words.txt")
#raw_other_words = other_words_file.read()
#other_words = raw_other_words.splitlines()
#print(other_words[:10])

#combine them
#words = sorted(answers + other_words)